Por conveniencia, copié el scripts ResultadosPrimeraVuelta.ipynb y aquí solo le cambio la tabla de entrada para que sean de datos de la segunda vuelta y cambio el nombre del archivo resultante. Esto para no escribir todo denuevo más que otra cosa.

In [35]:
import pandas as pd
import numpy as np
import pyreadstat as pyd

In [36]:
# datos de la primera vuelta electoral
primeravuelta, meta1 = pyd.read_sav('Data/Segunda-Vuelta.sav', apply_value_formats=True)

# Se analizó esta parroquia, que parece tener novedades en la tabla, por eso se exluye.
primeravuelta = primeravuelta[primeravuelta['PARROQUIA_CODIGO'] != 6150]

In [37]:
primeravuelta.columns

Index(['DIGNIDAD_CODIGO', 'DIGNIDAD_NOMBRE', 'PROVINCIA_CODIGO',
       'PROVINCIA_NOMBRE', 'CIRCUNSCRIPCION_CODIGO', 'CIRCUNSCRIPCION_NOMBRE',
       'CANTON_CODIGO', 'CANTON_NOMBRE', 'PARROQUIA_CODIGO',
       'PARROQUIA_NOMBRE', 'JUNTA_SEXO', 'OP_CODIGO', 'CANDIDATO_NOMBRE',
       'SUFRAGANTES', 'BLANCOS', 'NULOS', 'VOTOS'],
      dtype='object')

In [38]:
# Ajustar tipo de datos - convertir float primero, luego int
primeravuelta = primeravuelta.astype({
    'DIGNIDAD_NOMBRE': str,
    'PROVINCIA_CODIGO': float,  # Primero a float
    'PROVINCIA_NOMBRE': str,
    'CIRCUNSCRIPCION_CODIGO': str,
    'CIRCUNSCRIPCION_NOMBRE': str,
    'CANTON_CODIGO': float,  # Primero a float
    'CANTON_NOMBRE': str, 
    'PARROQUIA_CODIGO': float,  # Primero a float
    'PARROQUIA_NOMBRE': str,
    'JUNTA_SEXO': str,
    'OP_CODIGO': float,  # Primero a float
    'SUFRAGANTES': float, 
    'BLANCOS': float,
    'NULOS': float, 
    'VOTOS': float,
    'CANDIDATO_NOMBRE': str
})

# Luego convertir a int (sin decimales)
primeravuelta[['PROVINCIA_CODIGO', 'CANTON_CODIGO', 'PARROQUIA_CODIGO', 'OP_CODIGO']] = \
primeravuelta[['PROVINCIA_CODIGO', 'CANTON_CODIGO', 'PARROQUIA_CODIGO', 'OP_CODIGO']].astype(int)

In [39]:
primeravuelta.head(5)

,DIGNIDAD_CODIGO,DIGNIDAD_NOMBRE,PROVINCIA_CODIGO,PROVINCIA_NOMBRE,CIRCUNSCRIPCION_CODIGO,CIRCUNSCRIPCION_NOMBRE,CANTON_CODIGO,CANTON_NOMBRE,PARROQUIA_CODIGO,PARROQUIA_NOMBRE,JUNTA_SEXO,OP_CODIGO,CANDIDATO_NOMBRE,SUFRAGANTES,BLANCOS,NULOS,VOTOS
0,11.0,PRESIDENTA/E Y VICEPRESIDENTA/E,1,AZUAY,0.0,,260,CUENCA,285,BAÑOS,FEMENINO,500,DANIEL NOBOA AZIN,9888.0,70.0,907.0,5563.0
1,11.0,PRESIDENTA/E Y VICEPRESIDENTA/E,1,AZUAY,0.0,,260,CUENCA,285,BAÑOS,FEMENINO,1644,LUISA GONZALEZ,9888.0,70.0,907.0,3346.0
2,11.0,PRESIDENTA/E Y VICEPRESIDENTA/E,1,AZUAY,0.0,,260,CUENCA,285,BAÑOS,MASCULINO,500,DANIEL NOBOA AZIN,8416.0,44.0,646.0,4808.0
3,11.0,PRESIDENTA/E Y VICEPRESIDENTA/E,1,AZUAY,0.0,,260,CUENCA,285,BAÑOS,MASCULINO,1644,LUISA GONZALEZ,8416.0,44.0,646.0,2917.0
4,11.0,PRESIDENTA/E Y VICEPRESIDENTA/E,1,AZUAY,0.0,,260,CUENCA,730,CUMBE,FEMENINO,500,DANIEL NOBOA AZIN,2402.0,19.0,202.0,1435.0


In [40]:
primeravuelta[['PROVINCIA_CODIGO', 'CANTON_CODIGO', 'PARROQUIA_CODIGO', 'PARROQUIA_NOMBRE']].head(5)

,PROVINCIA_CODIGO,CANTON_CODIGO,PARROQUIA_CODIGO,PARROQUIA_NOMBRE
0,1,260,285,BAÑOS
1,1,260,285,BAÑOS
2,1,260,285,BAÑOS
3,1,260,285,BAÑOS
4,1,260,730,CUMBE


In [41]:
dic = pd.read_excel('data/diccionario_parroquias.xlsx')

In [42]:
# Renombrar columnas para que coincidan con primeravuelta
dic = dic.rename(columns={'CODPRO': 'PROVINCIA_CODIGO',
                          'CODCAN': 'CANTON_CODIGO',
                          'CODPAR': 'PARROQUIA_CODIGO'}
                ).astype({'PROVINCIA_CODIGO': int, 
                          'CANTON_CODIGO': int,
                          'PARROQUIA_CODIGO': int
                          })

In [43]:
primeravuelta = primeravuelta.merge(
    dic,
    on = ['PROVINCIA_CODIGO', 'CANTON_CODIGO', 'PARROQUIA_CODIGO'], how='left'
    )

De momento, exploraremos solo los votos presidenciales

In [44]:
# Filtrar datos para PRESIDENTE Y VICEPRESIDENTE
presidente1 = primeravuelta[primeravuelta['DIGNIDAD_NOMBRE'] == 'PRESIDENTA/E Y VICEPRESIDENTA/E']

Los resultados de sufragantes, nulos y blancos solo pueden obtenerse a nivel de parroquia y junta de sexo. Además, hay que recordar que como el mapa del CNE es más viejo, hay que tratar la base para luego homologarla con el censo del 2022.

Observa el ejemplo siguiente.

In [45]:
# Así deben desagregarse los datos
presidente1[presidente1['ADM3_PCODE'] == 'EC010150']\
    .groupby(['ADM3_PCODE', 'PARROQUIA_NOMBRE', 'JUNTA_SEXO'], as_index=False)\
    .agg(Sufragantes=('SUFRAGANTES', 'mean'),
         Votos_válidos=('VOTOS', 'sum'),
         Nulos=('NULOS', 'mean'),
         Blancos=('BLANCOS', 'mean')
        )

,ADM3_PCODE,PARROQUIA_NOMBRE,JUNTA_SEXO,Sufragantes,Votos_válidos,Nulos,Blancos
0,EC010150,MACHANGARA,FEMENINO,4924.0,4618.0,288.0,18.0
1,EC010150,MACHANGARA,MASCULINO,4223.0,3924.0,284.0,15.0


In [46]:
# exportar para verificar sumas:
# presidente1[presidente1['PARROQUIA_NOMBRE'] == 'CAÑARIBAMBA'].to_excel("data/comprobaciones/canaribamba.xlsx", index=False)

Aun así hay leves descuadres en las sumas de válidos, blancos y nulos. Pero eso ya es cuestión de la fuente. De momento, vamos a tratar los datos como sigue:

In [47]:
# Necesitamos un doble tratamiento, ya que ADM3_PCODE puede contener
# más de una parroquia lista en el CNE. (Esto puede comporbarse en la 
# unión espacial del cuaderno de trabajo homologar.ipynb).
# Así deben desagregarse los datos
resumen1 = presidente1\
                .groupby(['ADM3_PCODE', 'PARROQUIA_NOMBRE', 'JUNTA_SEXO'], as_index=False)\
                .agg(Sufragantes=('SUFRAGANTES', 'mean'),
                    Votos_válidos=('VOTOS', 'sum'),
                    Nulos=('NULOS', 'mean'),
                    Blancos=('BLANCOS', 'mean')
                    )

resumen1 = resumen1.groupby(['ADM3_PCODE', 'JUNTA_SEXO'], as_index=False)\
    .agg(Sufragantes=('Sufragantes', 'sum'),
         Votos_válidos=('Votos_válidos', 'sum'),
         Nulos=('Nulos', 'mean'),
         Blancos=('Blancos', 'mean')
         )

resumen1.head(10)

,ADM3_PCODE,JUNTA_SEXO,Sufragantes,Votos_válidos,Nulos,Blancos
0,EC010150,FEMENINO,4924.0,4618.0,288.0,18.0
1,EC010150,MASCULINO,4223.0,3924.0,284.0,15.0
2,EC010151,FEMENINO,9888.0,8909.0,907.0,70.0
3,EC010151,MASCULINO,8416.0,7725.0,646.0,44.0
4,EC010152,FEMENINO,2402.0,2182.0,202.0,19.0
5,EC010152,MASCULINO,1778.0,1655.0,107.0,16.0
6,EC010153,FEMENINO,524.0,472.0,40.0,12.0
7,EC010153,MASCULINO,500.0,440.0,48.0,12.0
8,EC010154,FEMENINO,966.0,860.0,102.0,4.0
9,EC010154,MASCULINO,696.0,617.0,72.0,7.0


In [48]:
# Calcular porcentajes de nulos y blancos
resumen1['Nulos_pct'] = resumen1['Nulos'] / resumen1['Sufragantes']
resumen1['Blancos_pct'] = resumen1['Blancos'] / resumen1['Sufragantes']
resumen1.head(10)

,ADM3_PCODE,JUNTA_SEXO,Sufragantes,Votos_válidos,Nulos,Blancos,Nulos_pct,Blancos_pct
0,EC010150,FEMENINO,4924.0,4618.0,288.0,18.0,0.058489,0.003656
1,EC010150,MASCULINO,4223.0,3924.0,284.0,15.0,0.067251,0.003552
2,EC010151,FEMENINO,9888.0,8909.0,907.0,70.0,0.091727,0.007079
3,EC010151,MASCULINO,8416.0,7725.0,646.0,44.0,0.076759,0.005228
4,EC010152,FEMENINO,2402.0,2182.0,202.0,19.0,0.084097,0.007910
5,EC010152,MASCULINO,1778.0,1655.0,107.0,16.0,0.060180,0.008999
6,EC010153,FEMENINO,524.0,472.0,40.0,12.0,0.076336,0.022901
7,EC010153,MASCULINO,500.0,440.0,48.0,12.0,0.096000,0.024000
8,EC010154,FEMENINO,966.0,860.0,102.0,4.0,0.105590,0.004141
9,EC010154,MASCULINO,696.0,617.0,72.0,7.0,0.103448,0.010057


# Distribución de votos a candidatos por sufragante

In [49]:
# Agregamos clasificaciones ideológicas de candidatos
Political_Compass = pd.read_excel('data/PiliticalCompass.xlsx')
Political_Compass.head(3)

,Candidato,CANDIDATO_NOMBRE,Eje Económico (Izq–Der),Eje Social (Lib–Aut),Clasificación,Justificación breve,Separación
0,Luisa González (RC),LUISA GONZALEZ,–2.0,+1.5,Izquierda autoritaria,"Estado fuerte, redistribución, liderazgo centr...",LUISA GONZALEZ\n(Izquierda autoritaria)
1,Jorge Escala (Unidad Popular),JORGE ESCALA,–2.5,+2.0,Izquierda autoritaria,"Marxismo sindical, estatismo, orden desde movi...",Izquierda autoritaria
2,Pedro Granja (PSE),PEDRO GRANJA,–2.0,+1.0,Izquierda autoritaria,"Socialismo democrático radical, impuestos a gr...",Izquierda autoritaria


In [50]:
presidente1clas = presidente1.merge(Political_Compass[['CANDIDATO_NOMBRE', 'Separación']], on='CANDIDATO_NOMBRE', how='left')

In [51]:
# Hay que hacer un doble tratamiento igual que antes
distribucioncandidatos  = \
    presidente1clas.groupby(['ADM3_PCODE', 'PARROQUIA_NOMBRE', 'JUNTA_SEXO', 'Separación'], 
                            as_index=False)\
                                .agg(Sufragantes=('SUFRAGANTES', 'mean'), 
                                     Votos_candidato=('VOTOS', 'sum'))

Existen descuadres en los resultados. Habría que considerarlo.

In [52]:
distribucioncandidatos2  = \
    distribucioncandidatos.groupby(['ADM3_PCODE', 'Separación', 'JUNTA_SEXO'], 
                            as_index=False)\
                                .agg(Sufragantes=('Sufragantes', 'mean'), 
                                     Votos_candidato=('Votos_candidato', 'sum'))

In [53]:
distribucioncandidatos2.head()
# Comprobar con distribucioncandidatos2.to_excel("data/comprobaciones/distribucioncandidatos_ec010150_sexo.xlsx", index=False)
# Recuerda filtrar un ADM3_PCODE

,ADM3_PCODE,Separación,JUNTA_SEXO,Sufragantes,Votos_candidato
0,EC010150,DANIEL NOBOA AZIN\n(Derecha liberal \nmoderada),FEMENINO,4924.0,3052.0
1,EC010150,DANIEL NOBOA AZIN\n(Derecha liberal \nmoderada),MASCULINO,4223.0,2498.0
2,EC010150,LUISA GONZALEZ\n(Izquierda autoritaria),FEMENINO,4924.0,1566.0
3,EC010150,LUISA GONZALEZ\n(Izquierda autoritaria),MASCULINO,4223.0,1426.0
4,EC010151,DANIEL NOBOA AZIN\n(Derecha liberal \nmoderada),FEMENINO,9888.0,5563.0


In [54]:
distribucioncandidatos2['Votos_candidato_pct'] = distribucioncandidatos2['Votos_candidato'] / distribucioncandidatos2['Sufragantes']

In [55]:
distribucioncandidatos2.head(4)

,ADM3_PCODE,Separación,JUNTA_SEXO,Sufragantes,Votos_candidato,Votos_candidato_pct
0,EC010150,DANIEL NOBOA AZIN\n(Derecha liberal \nmoderada),FEMENINO,4924.0,3052.0,0.619821
1,EC010150,DANIEL NOBOA AZIN\n(Derecha liberal \nmoderada),MASCULINO,4223.0,2498.0,0.591523
2,EC010150,LUISA GONZALEZ\n(Izquierda autoritaria),FEMENINO,4924.0,1566.0,0.318034
3,EC010150,LUISA GONZALEZ\n(Izquierda autoritaria),MASCULINO,4223.0,1426.0,0.337675


In [56]:
reestructurado = distribucioncandidatos2.\
    pivot(index=['ADM3_PCODE', 'JUNTA_SEXO'], columns=['Separación'], 
          values='Votos_candidato_pct').fillna(0).reset_index()

In [57]:
# Unir tablas
table_resumen1 = resumen1.merge(reestructurado, on=['ADM3_PCODE', 'JUNTA_SEXO'], how='left').fillna(0)

In [58]:
# Separamos por sexo y exportamos (porque será una estructura más manejable en Tableau)
table_resumen1[table_resumen1['JUNTA_SEXO'] == 'MASCULINO'].to_csv('data/2daM.csv', index=False, decimal=',', sep=';')
table_resumen1[table_resumen1['JUNTA_SEXO'] == 'FEMENINO'].to_csv('data/2daF.csv', index=False, decimal=',', sep=';')
#table_resumen1.to_csv('data/ResultadosSegundaVuelta.csv', index=False, decimal=',', sep=';')